<a href="https://colab.research.google.com/github/DanielaSchacherer/IDC-Tutorials/blob/bmdeep_tutorial/notebooks/pathomics/BMDeep_collection_snippet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BoneMarrowWSI-PediatricLeukemia


## Background

This notebook introduces the `BoneMarrowWSI-PediatricLeukemia` collection, which is presented in [this preprint](https://www.arxiv.org/pdf/2509.15895) and was recently added to IDC.

- **Images**: The `BoneMarrowWSI-PediatricLeukemia` dataset comprises bone marrow aspirate smear WSIs for 246 pediatric cases of leukemia, including acute lymphoid leukemia (ALL), acute myeloid leukemia (AML), and chronic myeloid leukemia (CML). The smears were prepared for the initial diagnosis (i.e., without prior treatment), stained in accordance with the Pappenheim method, and scanned at 40x magnification.
- **Annotations**: The images have been annotated with rectangular regions of interest (ROI) of the evaluable monolayer area and a total of 45176 cell bounding box annotations have been placed (with few exceptions) within the ROIs. For a subset of 232 ROIs all cells and other haematological structures have been labelled by multiple experts in a consensus labeling approach with 49 distinct (cell type) classes. The consensus labelling approach worked as follows: each bounding box was successively labelled by different experts in so-called "annotation sessions" until (a) the bounding box has been labelled by at least two experts, and (b) the most frequent label constitues at least half of all labels given to that bounding box (and is then termed "consensus class"). In summary, the following annotations are available:  

    - For each slide: ROI annotations of the monolayer area for each slide
    - For some slides: Unlabeled cell bounding boxes
    - For some slides: Cell bounding boxes with cell type labels for each annotation session plus the finally obtained consensus.

This notebook concentrates on **how to access and work with the annotation data**, that are made available in DICOM Microscopy Bulk Simple Annotation format. As a general introduction to this format, we recommend having a look at [this tutorial notebook](https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/pathomics/microscopy_dicom_ann_intro.ipynb).


<img src="https://raw.githubusercontent.com/ImagingDataCommons/IDC-Tutorials/master/notebooks/pathomics/bmdeep_annotations_example.png" alt="Example visualization of annotations" width="1000"/>



## Prerequisites
**Installations**
* **Install highdicom:** [highdicom](https://highdicom.readthedocs.io/en/latest/introduction.html) was specifically designed to work with DICOM objects holding image-derived information, e.g. annotations and measurements. Detailed information on highdicom's functionality can be found in its [user guide](https://highdicom.readthedocs.io/en/latest/usage.html).
* **Install wsidicom:** The [wsidicom](https://pypi.org/project/wsidicom/) Python package provides functionality to open and extract image or metadata from WSIs.
* **Install idc-index:** The Python package [idc-index](https://pypi.org/project/idc-index/) facilitates queries of the basic metadata and download of DICOM files hosted by the IDC.

In [ ]:
%%capture
!pip install highdicom
!pip install wsidicom
!pip install idc-index --upgrade

## Imports

In [ ]:
import highdicom as hd
from idc_index import index
import pandas as pd
from collections import defaultdict
from google.cloud import storage
from pathlib import Path
from typing import List, Union, Tuple

## Finding the `BoneMarrowWSI-PediatricLeukemia` dataset on IDC
To access and download image and ANNs files, we utilize the Python package [idc-index](https://github.com/ImagingDataCommons/idc-index).

In [ ]:
idc_client = index.IDCClient() # set-up idc_client
idc_client.fetch_index('sm_instance_index')

First, we verify that we have indeed 246 WSI (=distinct StudyInstanceUIDs) in the `BoneMarrowWSI-PediatricLeukemia` collection:

In [ ]:
query_slide_count = '''
SELECT COUNT(DISTINCT StudyInstanceUID)
FROM
    index
WHERE
    collection_id = 'bonemarrowwsi_pediatricleukemia' AND Modality='SM'
'''
print(idc_client.sql_query(query_slide_count))

   count(DISTINCT StudyInstanceUID)
0                               246


Next, let's have a look on the available annotation (ANN) files:

In [ ]:
query_anns = '''
SELECT
    SeriesDescription,
    SeriesInstanceUID,
    ARRAY_AGG(StudyInstanceUID) AS StudyInstanceUID,
    ARRAY_AGG(Modality) AS Modality
FROM
    index
WHERE
    collection_id = 'bonemarrowwsi_pediatricleukemis' AND Modality='ANN'
GROUP BY
    SeriesInstanceUID,
    SeriesDescription
ORDER BY
    StudyInstanceUID
'''
annotations = idc_client.sql_query(query_anns)
print(annotations)

Empty DataFrame
Columns: [SeriesDescription, SeriesInstanceUID, StudyInstanceUID, Modality]
Index: []


We can see, that for each slide (i.e. DICOM Study) there are multiple ANN Series. Looking at the SeriesDescription, we can assert what is described in the Background section of this notebook.


*   Each slide has "Monolayer regions of interest for cell classification" annotations.
*   For some slides, there is one ANN Series with "Unlabeled cell bounding boxes", while for others, there are multiple ANN Series containing "Cell bounding boxes with cell type labels" for different annotation sessions or the consensus labels.



## Viewing annotations


Annotations can also be viewed and explored in detail on its respective slide using the Slim viewer. In the Slim viewer's interface at the bottom of the right sidebar you may select the ANN Series of interest to you from the drop-down menue, then click on `Annotation Groups` and switch the slider(s) to make annotations visible.

In [ ]:
viewer_url = idc_client.get_viewer_URL(studyInstanceUID=annotations['StudyInstanceUID'].iloc[0][0], viewer_selector='slim')
from IPython.display import IFrame
IFrame(viewer_url, width=1500, height=900)

## Accessing annotations

Since the annotation dataset is of reasonable size it could be downloaded completely and accessed from the local disk. However, a more desirable approach especially for larger size datasets is to directly extract the relevant information from the objects in the cloud. The following functions `get_roi_annotations()`, `get_unlabeled_cell_annotations()` and `get_labeled_cell_annotations()` can be used for this approach.

Note, that the selection of the respective files, i.e. files containing ROI annotations, labeled or unlabeled cell annotations, is done by filtering for the respective SeriesDescription.

In [ ]:
def get_roi_annotations():
    query_roi_anns = '''
    SELECT
        SeriesInstanceUID
    FROM
        index
    WHERE
        collection_id = 'bonemarrowwsi_pediatricleukemis'
        AND Modality='ANN'
        AND LOWER(SeriesDescription) LIKE '%monolayer%'
    ORDER BY
        StudyInstanceUID
    '''
    roi_series = idc_client.sql_query(query_roi_anns)
    rois = extract_rois(roi_series['SeriesInstanceUID'].tolist())
    return rois

In [ ]:
def extract_rois(series_uids: List[str]) -> pd.DataFrame:
    rows = []
    for series_uid in series_uids:
        file_urls = idc_client.get_series_file_URLs(seriesInstanceUID=series_uid, source_bucket_location="gcs")
        for file_url in file_urls:
            ann = hd.ann.annread(file_url)
            for ann_group in ann.get_annotation_groups():
                coords = ann_group.get_graphic_data(coordinate_type='2D')
                m_names, m_values, m_units = ann_group.get_measurements()
                for c, m in zip(coords, m_values):
                    rows.append({
                        'reference_series_id': ann.ReferencedSeriesSequence[0].SeriesInstanceUID,
                        'reference_sop_id': ann.ReferencedImageSequence[0].ReferencedSOPInstanceUID,
                        'roi_id': int(m[0]), # allow empty roi_id,
                        'roi_label': ann_group.label,
                        'roi_coordinates': c
                    })
    rois = pd.DataFrame(rows)
    return rois

In [ ]:
rois = get_roi_annotations()
display(rois)

In [ ]:
def get_labeled_cells():
    query_cell_anns_labeled = '''
    SELECT
        SeriesInstanceUID
    FROM
        index
    WHERE
        collection_id = 'bonemarrowwsi_pediatricleukemis'
        AND Modality='ANN'
        AND LOWER(SeriesDescription) LIKE '%labels%'
    ORDER BY
        StudyInstanceUID
    '''
    labeled_cell_series = idc_client.sql_query(query_cell_anns_labeled)
    labeled_cells = extract_cells(labeled_cell_series['SeriesInstanceUID'].tolist())
    return labeled_cells

In [ ]:
def get_unlabeled_cells():
    query_cell_anns_unlabeled = '''
    SELECT
        SeriesInstanceUID
    FROM
        index
    WHERE
        collection_id = 'bonemarrowwsi_pediatricleukemis'
        AND Modality='ANN'
        AND LOWER(SeriesDescription) LIKE '%unlabeled%'
    ORDER BY
        StudyInstanceUID
    '''
    unlabeled_cell_series = idc_client.sql_query(query_cell_anns_unlabeled)
    unlabeled_cells = extract_cells(unlabeled_cell_series['SeriesInstanceUID'].tolist())
    return unlabeled_cells

In [ ]:
def get_annotation_session(ann: hd.ann.sop.MicroscopyBulkSimpleAnnotations) -> str:
    return ann.SeriesDescription.split(':')[0]

In [ ]:
def extract_cells(series_uids: List[str]) -> pd.DataFrame:
    rows = []
    for series_uid in series_uids:
        file_urls = idc_client.get_series_file_URLs(seriesInstanceUID=series_uid, source_bucket_location="gcs")
        for file_url in file_urls:
            ann = hd.ann.annread(file_url)
            for ann_group in ann.get_annotation_groups():
                coords = ann_group.get_graphic_data(coordinate_type='2D')
                m_names, m_values, m_units = ann_group.get_measurements()
                for c, m in zip(coords, m_values):
                    rows.append({
                        'reference_series_id': ann.ReferencedSeriesSequence[0].SeriesInstanceUID,
                        'reference_sop_id': ann.ReferencedImageSequence[0].ReferencedSOPInstanceUID,
                        'annotation_session': get_annotation_session(ann),
                        'cell_id': int(m[0]),
                        'roi_id': int(m[1]) if m.size > 1 else None, # allow empty roi_id,
                        'cell_label': ann_group.label,
                        'cell_coordinates': c
                    })
    cells = pd.DataFrame(rows)
    return cells

In [ ]:
unlabeled_cells = get_unlabeled_cells()
display(unlabeled_cells)

In [ ]:
labeled_cells = get_labeled_cells()
display(labeled_cells)

## Test with private bucket

In [ ]:
# for local access
def extract_rois(series_uids: List[str]) -> pd.DataFrame:
    rows = []
    for file in os.listdir('./downloaded_files'):
        if 'roi' in file:
            ann = hd.ann.annread('./downloaded_files/'+ file)
            for ann_group in ann.get_annotation_groups():
                coords = ann_group.get_graphic_data(coordinate_type='2D')
                m_names, m_values, m_units = ann_group.get_measurements()
                for c, m in zip(coords, m_values):
                    rows.append({
                        'reference_series_id': ann.ReferencedSeriesSequence[0].SeriesInstanceUID,
                        'reference_sop_id': ann.ReferencedImageSequence[0].ReferencedSOPInstanceUID,
                        'roi_id': int(m[0]), # allow empty roi_id,
                        'roi_label': ann_group.label,
                        'roi_coordinates': c
                    })
    rois = pd.DataFrame(rows)
    return rois

display(extract_rois(None))

,reference_series_id,reference_sop_id,roi_id,roi_label,roi_coordinates
0,1.2.826.0.1.3680043.8.498.81829257432420839308...,1.2.826.0.1.3680043.8.498.88303719051231458307...,2,region_of_interest,"[[63021.0, 166331.0], [65069.0, 166331.0], [65..."
1,1.2.826.0.1.3680043.8.498.81829257432420839308...,1.2.826.0.1.3680043.8.498.88303719051231458307...,3,region_of_interest,"[[128929.0, 184949.0], [130977.0, 184949.0], [..."
2,1.2.826.0.1.3680043.8.498.81829257432420839308...,1.2.826.0.1.3680043.8.498.88303719051231458307...,2002,region_of_interest,"[[44298.0, 145648.0], [75522.0, 145648.0], [75..."
3,1.2.826.0.1.3680043.8.498.77272910370076975556...,1.2.826.0.1.3680043.8.498.50197427852324449642...,82,region_of_interest,"[[8235.0, 164826.0], [10283.0, 164826.0], [102..."
4,1.2.826.0.1.3680043.8.498.77272910370076975556...,1.2.826.0.1.3680043.8.498.50197427852324449642...,83,region_of_interest,"[[134067.0, 152341.0], [136115.0, 152341.0], [..."
5,1.2.826.0.1.3680043.8.498.77272910370076975556...,1.2.826.0.1.3680043.8.498.50197427852324449642...,2195,region_of_interest,"[[87180.0, 160884.0], [95250.0, 160884.0], [95..."
6,1.2.826.0.1.3680043.8.498.77272910370076975556...,1.2.826.0.1.3680043.8.498.50197427852324449642...,2196,region_of_interest,"[[12720.0, 162063.0], [23165.0, 162063.0], [23..."
7,1.2.826.0.1.3680043.8.498.95463846645682241501...,1.2.826.0.1.3680043.8.498.13783623809302100421...,1028,region_of_interest,"[[99052.0, 80470.0], [101100.0, 80470.0], [101..."
8,1.2.826.0.1.3680043.8.498.95463846645682241501...,1.2.826.0.1.3680043.8.498.13783623809302100421...,1029,region_of_interest,"[[80013.0, 88503.0], [82061.0, 88503.0], [8206..."


In [ ]:
## for local access
import os
def get_cells(series_uids: List[str], annotation_session: Union[int,str]) -> pd.DataFrame:
    rows = []
    for file in os.listdir('./downloaded_files'):
        if 'cell' in file:
            ann = hd.ann.annread('./downloaded_files/'+ file)
            for ann_group in ann.get_annotation_groups():
                coords = ann_group.get_graphic_data(coordinate_type='2D')
                m_names, m_values, m_units = ann_group.get_measurements()
                for c, m in zip(coords, m_values):
                    rows.append({
                        'reference_series_id': ann.ReferencedSeriesSequence[0].SeriesInstanceUID,
                        'reference_sop_id': ann.ReferencedImageSequence[0].ReferencedSOPInstanceUID,
                        'annotation_session': ann.SeriesDescription,
                        'cell_id': int(m[0]),
                        'roi_id': int(m[1]) if m.size > 1 else None, # allow empty roi_id,
                        'cell_label': ann_group.label,
                        'cell_coordinates': c
                    })
    cells = pd.DataFrame(rows)#, columns=['series_id', 'sop_id', 'cell_id', 'roi_id', 'cell_label', 'cell_coordinates'])
    return cells

display(get_cells(None, None))

,reference_series_id,reference_sop_id,annotation_session,cell_id,roi_id,cell_label,cell_coordinates
0,1.2.826.0.1.3680043.8.498.77272910370076975556...,1.2.826.0.1.3680043.8.498.50197427852324449642...,Session 4: Cell bounding boxes with cell type ...,12087,82.0,no_consensus_found,"[[9512.0, 166724.0], [9633.0, 166724.0], [9633..."
1,1.2.826.0.1.3680043.8.498.77272910370076975556...,1.2.826.0.1.3680043.8.498.50197427852324449642...,Session 4: Cell bounding boxes with cell type ...,12121,82.0,no_consensus_found,"[[9964.0, 166318.0], [10068.0, 166318.0], [100..."
2,1.2.826.0.1.3680043.8.498.77272910370076975556...,1.2.826.0.1.3680043.8.498.50197427852324449642...,Session 4: Cell bounding boxes with cell type ...,12124,82.0,no_consensus_found,"[[9656.0, 166380.0], [9766.0, 166380.0], [9766..."
3,1.2.826.0.1.3680043.8.498.77272910370076975556...,1.2.826.0.1.3680043.8.498.50197427852324449642...,Session 4: Cell bounding boxes with cell type ...,12125,82.0,no_consensus_found,"[[9663.0, 166482.0], [9771.0, 166482.0], [9771..."
4,1.2.826.0.1.3680043.8.498.77272910370076975556...,1.2.826.0.1.3680043.8.498.50197427852324449642...,Session 4: Cell bounding boxes with cell type ...,12126,82.0,no_consensus_found,"[[9748.0, 166496.0], [9860.0, 166496.0], [9860..."
...,...,...,...,...,...,...,...
1544,1.2.826.0.1.3680043.8.498.77272910370076975556...,1.2.826.0.1.3680043.8.498.50197427852324449642...,Session 3: Cell bounding boxes with cell type ...,12126,82.0,no_consensus_found,"[[9748.0, 166496.0], [9860.0, 166496.0], [9860..."
1545,1.2.826.0.1.3680043.8.498.77272910370076975556...,1.2.826.0.1.3680043.8.498.50197427852324449642...,Session 3: Cell bounding boxes with cell type ...,12138,83.0,no_consensus_found,"[[133938.0, 153263.0], [134073.0, 153263.0], [..."
1546,1.2.826.0.1.3680043.8.498.77272910370076975556...,1.2.826.0.1.3680043.8.498.50197427852324449642...,Session 3: Cell bounding boxes with cell type ...,12142,83.0,no_consensus_found,"[[135411.0, 152959.0], [135516.0, 152959.0], [..."
1547,1.2.826.0.1.3680043.8.498.77272910370076975556...,1.2.826.0.1.3680043.8.498.50197427852324449642...,Session 3: Cell bounding boxes with cell type ...,12174,83.0,no_consensus_found,"[[134331.0, 152250.0], [134479.0, 152250.0], [..."


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project='idc-pathomics-000')

query_anns = '''
SELECT
    SeriesDescription,
    SeriesInstanceUID,
    ARRAY_AGG(StudyInstanceUID) AS StudyInstanceUID,
    ARRAY_AGG(Modality) AS Modality
FROM
    `idc-pathomics-000.BMDeep_annotations.dicom_all`
WHERE
    Modality='ANN'
GROUP BY
    SeriesInstanceUID,
    SeriesDescription
'''
annotations = client.query(query_anns).result()
for row in annotations:
    print(row)

In [ ]:
query_roi_anns = '''
SELECT
    SeriesInstanceUID
FROM
    `idc-pathomics-000.BMDeep_annotations.dicom_all`
WHERE
    --collection_id = 'bonemarrowwsi_pediatricleukemis'
    Modality='ANN'
    AND LOWER(SeriesDescription) LIKE '%unlabeled%'
ORDER BY
    StudyInstanceUID
'''
roi_annotations = client.query(query_roi_anns)
for row in roi_annotations:
    print(row)

Row(('1.2.826.0.1.3680043.10.511.3.62471311052754691342852119271665952',), {'SeriesInstanceUID': 0})


In [ ]:
from google.cloud import storage
import os

def download_all_files_from_bucket(bucket_name, destination_folder):
    client = storage.Client()  # Make sure credentials are configured (e.g., GOOGLE_APPLICATION_CREDENTIALS)
    bucket = client.bucket(bucket_name)

    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    blobs = bucket.list_blobs()
    for blob in blobs:
        file_path = os.path.join(destination_folder, blob.name)
        os.makedirs(os.path.dirname(file_path), exist_ok=True)  # Create directories as needed
        blob.download_to_filename(file_path)
        print(f"Downloaded {blob.name} to {file_path}")

# Usage
bucket_name = "bmdeep_anns"
destination_folder = "./downloaded_files"
download_all_files_from_bucket(bucket_name, destination_folder)


Downloaded 02E74F10E0327AD868D138F2B4FDD6F0_1_bm_cells_ann_session_1.dcm to ./downloaded_files/02E74F10E0327AD868D138F2B4FDD6F0_1_bm_cells_ann_session_1.dcm
Downloaded 02E74F10E0327AD868D138F2B4FDD6F0_1_bm_cells_ann_session_2.dcm to ./downloaded_files/02E74F10E0327AD868D138F2B4FDD6F0_1_bm_cells_ann_session_2.dcm
Downloaded 02E74F10E0327AD868D138F2B4FDD6F0_1_bm_cells_ann_session_3.dcm to ./downloaded_files/02E74F10E0327AD868D138F2B4FDD6F0_1_bm_cells_ann_session_3.dcm
Downloaded 02E74F10E0327AD868D138F2B4FDD6F0_1_bm_cells_ann_session_4.dcm to ./downloaded_files/02E74F10E0327AD868D138F2B4FDD6F0_1_bm_cells_ann_session_4.dcm
Downloaded 02E74F10E0327AD868D138F2B4FDD6F0_1_bm_cells_ann_session_5.dcm to ./downloaded_files/02E74F10E0327AD868D138F2B4FDD6F0_1_bm_cells_ann_session_5.dcm
Downloaded 02E74F10E0327AD868D138F2B4FDD6F0_1_bm_cells_ann_session_consensus.dcm to ./downloaded_files/02E74F10E0327AD868D138F2B4FDD6F0_1_bm_cells_ann_session_consensus.dcm
Downloaded 02E74F10E0327AD868D138F2B4FDD6F